In [ ]:
import pandas as pd
import re
import os

In [ ]:

# folder path
dir_path = r'../csv/all_issues' ## Point to the extracted folder containing all the issues csv files

# list to store files
res = []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path)
print(res)

In [ ]:

# Regular expression to capture various variations of "high priority"
high_priority = r"\b(?:priority[\s\-_,.]*high|high[\s\-_,.]*priority|top[\s\-_,.]*priority|urgent|critical|high[\s\-_,.]*prio|high[\s\-_,.]*pri|prio[\s\-_,.]*high|pri[\s\-_,.]*high)\b"

medium_priority = r"\b(?:priority[\s\-_,.]*medium|medium[\s\-_,.]*priority|moderate[\s\-_,.]*priority|medium[\s\-_,.]*prio|medium[\s\-_,.]*pri|prio[\s\-_,.]*medium|pri[\s\-_,.]*medium)\b"

low_priority = r"\b(?:priority[\s\-_,.]*low|low[\s\-_,.]*priority|minor[\s\-_,.]*priority|low[\s\-_,.]*prio|low[\s\-_,.]*pri|prio[\s\-_,.]*low|pri[\s\-_,.]*low)\b"



# Define the patterns
patterns = ["important", "urgent", "critical"]

# Initialize a dictionary to store DataFrames for each pattern
pattern_dataframes = {pattern: [] for pattern in patterns}

for i in res:
    try:
        file_path = f"{dir_path}/{i}"
        df = pd.read_csv(file_path, index_col=0)

        # Apply filtering for each pattern
        for pattern in patterns:
            filtered_df = df[df["labels"].str.contains(pattern, case=False, na=False, regex=True)]
            if not filtered_df.empty:
                pattern_dataframes[pattern].append(filtered_df)

        print(f"Processed: {i}")
    except Exception as e:
        print(f"Error processing {i}: {e}")

# Concatenate DataFrames for each pattern
for pattern in patterns:
    if pattern_dataframes[pattern]:
        pattern_dataframes[pattern] = pd.concat(pattern_dataframes[pattern], ignore_index=True)
    else:
        pattern_dataframes[pattern] = pd.DataFrame()  # Create an empty DataFrame if no data was found


In [ ]:
pattern_dataframes["critical"].labels.value_counts().to_frame()[:50]

In [ ]:
# Assuming 'appended_data' is your DataFrame
label_repo_counts = pattern_dataframes["critical"].groupby(['labels', 'repo']).size().reset_index(name='counts')
top_label_repo = label_repo_counts.sort_values(by='counts', ascending=False).head(60)
top_label_repo


In [ ]:
# Convert to csv
# data_to_csv = f"csv/final_medium_dataset.csv"
# appended_data.to_csv(data_to_csv)# type: ignore